In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.image as mpimg
import cv2
import os
import glob
from skimage import io, transform
from tensorflow.keras import layers, models, losses

In [2]:
# to read created ecg images
Default_path = '/.'

# Call the function that reads the picture to get the dataset of pictures and labels
def read_img(path=Default_path, w=1200, h=400, c=3):
    cate = [path + x for x in os.listdir(path) if os.path.isdir(path + x)]
    imgs = []
    labels = []
    for idx, folder in enumerate(cate):
        for im in glob.glob(folder + '/*.jpg'):
            # print('reading the images:%s' % (im))
            img = cv2.imread(im)
            img = cv2.resize(img, (w, h))
            imgs.append(img)
            labels.append(idx)
    
    print("ECG Images Read!!!")
    return np.array(imgs), np.asarray(labels, np.int32)


# Resize all pics
w = 1800
h = 600
c = 3
path = './ptbdb/'

In [3]:
data, label = read_img(path=path, w=w, h=h, c=c) # call the function to read the data

ECG Images Read!!!


In [4]:
# Create training and testing dataset

num_example = data.shape[0]  # the total number of pictures
arr = np.arange(num_example)  # np.arange (start value, end value, step size)
np.random.shuffle(arr)  # Rearrange the assignment after scrambling
data = data[arr]
label = label[arr]

from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(data, label, test_size = 0.2) 

In [5]:
# to shuffle the order in which data to be fed for training
num_example = train_images.shape[0]  # the total number of pictures
arr1 = np.arange(num_example)  # np.arange (start value, end value, step size)
np.random.shuffle(arr1)  # Rearrange the assignment after scrambling
train_images = train_images[arr1]
train_labels = train_labels[arr1]

# to shuffle the order in which data to be fed for testing
num_example = test_images.shape[0]  # the total number of pictures
arr2 = np.arange(num_example)  # np.arange (start value, end value, step size)
np.random.shuffle(arr2)  # Rearrange the assignment after scrambling
test_images = test_images[arr2]
test_labels = test_labels[arr2]

In [ ]:
#%% Create the convolutional base 
    
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (w, h, c)))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

# cnn_model.summary()

#%% Add Dense layers on top

cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(64, activation = 'relu'))
cnn_model.add(layers.Dense(number_of_classes))

cnn_model.summary()

In [ ]:
#%% Compile and train (fit) the model

cnn_model.compile(optimizer = 'adam',
              loss = losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = cnn_model.fit(train_images, train_labels, epochs = number_of_iterations, 
                    validation_data = (test_images, test_labels))


In [ ]:
#%% Evaluate the model

plt.plot(history.history['accuracy'], label = 'Training_accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
plt.legend(loc = 'lower right')

test_loss, test_acc = cnn_model.evaluate(test_images,  test_labels, verbose = 2)

print('\nTest accuracy:', test_acc)

In [ ]:
# Save the entire model as a SavedModel.
cnn_model.save(model_path) 